In [ ]:
from keras.datasets import mnist
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.models import Sequential, Model



C:\Users\LENOVO\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
(X_train,_),(_,_)=mnist.load_data()

In [ ]:
print(X_train.shape)

In [ ]:
X_train=X_train.reshape(60000,-1)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
X_train=(X_train.astype('float32')-127.5)/127.5

In [ ]:
import numpy as np
np.max(X_train)

In [ ]:
np.min(X_train)

In [ ]:
noise_size=10

In [ ]:
##generator
generator=Sequential()
generator.add(Dense(256,input_dim=noise_size))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784,activation="tanh"))
generator.compile(loss="binary_crossentropy",optimizer=Adam(lr=2e-4,beta_1=0.5))
generator.summary()

In [ ]:
discriminator = Sequential()
discriminator.add(Dense(512,input_shape=(784,)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1,activation="tanh"))
##sigmoid
discriminator.compile(loss="binary_crossentropy",optimizer=Adam(lr=2e-4,beta_1=0.5))
discriminator.summary()

In [ ]:
discriminator.trainable=False
gan_input=Input(shape=(noise_size,))
y_out=generator(gan_input)
gan_output=discriminator(y_out)
model=Model(gan_input,gan_output)
model.compile(loss="binary_crossentropy",optimizer=Adam(lr=2e-4,beta_1=0.5))

In [ ]:
def print_img(samples=100):
    noise=np.random.normal(0,1,size=(samples,10))
    imgs=generator.predict(noise)
    imgs=imgs.reshape(samples,28,28)
    plt.figure(figsize=(10,10))
    for i in range(samples):
        plt.subplot(10,10,i+1)
        plt.axis("off")
        plt.imshow(imgs[i],cmap="gray")
    plt.show()
    ##cmap=gray
    plt.tight_layout()
         
        

In [ ]:
import random
epochs=10000
total_loss=[]
loss=[]
for epoch in range(epochs):
    ##step1
    half_batch=int(X_train.shape[0]/2)
    noise=np.random.normal(0,1,size=(half_batch,noise_size))
    fake_img=generator.predict(noise)
    idx=np.random.randint(0,X_train.shape[0],half_batch)
    real_img=X_train[idx]
    
    ###one sided label smoothing
    real_y=np.ones((half_batch,1))*0.9
    fake_y=np.zeros((half_batch,1))
    d_loss_f=discriminator.train_on_batch(fake_img,fake_y)
    d_loss_r=discriminator.train_on_batch(real_img,real_y) 
    total_loss.append(d_loss_f*0.5+d_loss_r*0.5)
    ##step2
    noise=np.random.normal(0,1,(X_train.shape[0],noise_size))
    ground_truth=np.ones((X_train.shape[0],1))
    g_loss=model.train_on_batch(noise,ground_truth)
    loss.append(g_loss)
    
    if(epoch+1)%250==0:
        print("generator %.2f discriminator %.2f" %(g_loss,d_loss_f*0.5+d_loss_r*0.5))
        print_img()
        


